O dataset contém dados de 4000 talhões. Um talhão é uma área de cana plantada, como se fosse um "quarteirão" de cana com tamanho arbitrário.
Os atributos são:
* chavesig: identificador do talhao
* tch: toneladas de cana por hectare (o que queremos prever)
* ndvi_a_p05: indice de vegetação no 5º mês após o plantio
* ncorte: número do corte
* linhas_falha_km: quantos km de falhas nas linhas de cana do talhao
* chuva_dia_pN: média de chuva por dia no mes N antes da colheita
* bal_hidrico_pN: balanço hídrico no mes N antes da colheita
* espacamento_m: espacamento entre as linhas de cana
* area_esti: area estimada do talhaoa

Com base nestas informações, construa um modelo capaz de prever, a quantidade de tch que sera colhida com base nos atributos que julgar necessários.
É simples, não precisa de nada complicado. Quero ver como vc pensa em fazer isso e como apresenta a linha de raciocínio. O pipeline de trabalho é bem similar ao que tem no script de casas, mas é do zero agora.

### Importando os dados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
base = pd.read_csv('sample_train_dataset_tch_pred.csv')

In [ ]:
# Apresentando o cabeçalho dos dados
base.head()

#### Implementação: Normalização/Desnormalização

In [ ]:
tch_y = base['tch']

#Normalizando os dados
normalized = map(lambda x: (x - tch_y.min())/( tch_y.max()-tch_y.min()), tch_y)
normalized = pd.Series(normalized)

#Desnormalizando os dados
unnormalized = map(lambda y: tch_y.min() + (y*(tch_y.max() - tch_y.min()) ), normalized)
unnormalized = pd.Series(unnormalized)


In [ ]:
#Analisando o número de linhas e colunas
base.shape

In [ ]:
# Verificando se há linhas null no dataset
base.isnull().sum()

In [ ]:
#descreve meus dados atraves de metricas
base.describe()

In [ ]:
#Lista com as colunas do dataset
base.columns

## Utilizando o método Regressão Múltipla


### Selecionando apenas as colunas necessárias do dataset

In [ ]:
colunas = ['tch','ndvi_a_p05', 'ncorte', 'areaesti','linhas_falha_km', 'chuva_dia_p04', 'chuva_dia_p12',
       'bal_hidrico_p12', 'bal_hidrico_p04', 'espacamento_m']

### Selecionando as variáveis independentes X

In [ ]:
colunas_x = ['ndvi_a_p05', 'ncorte', 'areaesti','linhas_falha_km', 'chuva_dia_p04', 'chuva_dia_p12',
       'bal_hidrico_p12', 'bal_hidrico_p04', 'espacamento_m']

In [ ]:
# Chamando o arquivo csv apenas com as colunas necessarias
base = pd.read_csv('sample_train_dataset_tch_pred.csv', usecols = colunas)


### Normalizando os dados(alterando os dados para uma escala comum)

***Objetivo:*** Alterar os valores das colunas numéricas no conjunto de dados para uma escala comum, sem distorcer as diferenças nos intervalos de valores



In [ ]:
#importando biblioteca para normalizar os dados
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Normalizando os dados para as variaveis indepedentes
x_scal = MinMaxScaler()
base[['ndvi_a_p05', 'ncorte', 'areaesti','linhas_falha_km', 'chuva_dia_p04', 'chuva_dia_p12',
       'bal_hidrico_p12', 'bal_hidrico_p04', 'espacamento_m']] = x_scal.fit_transform(base[['ndvi_a_p05', 'ncorte', 'areaesti','linhas_falha_km', 'chuva_dia_p04', 'chuva_dia_p12',
       'bal_hidrico_p12', 'bal_hidrico_p04', 'espacamento_m']])

In [ ]:
base.head()

In [ ]:
# Normalizando as variaveis para y
y_scal = MinMaxScaler()
base[['tch']] = y_scal.fit_transform(base[['tch']])


In [ ]:
# y recebendo a variavel depedente tch
y = base['tch']

In [ ]:
# x recebendo a lista com as variaveis indepedentes de x
x = base.drop('tch', axis = 1)

### Dividindo os dados

In [ ]:
#featues colunas
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
colunas_x = [tf.feature_column.numeric_column(key = c) for c in colunas_x]

In [ ]:
#Dividindo a base de dados em teste e train para o modelo
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x,y, test_size = 0.3)

### Definido função para treinamento

In [ ]:
# Function train
function_treinamento = tf.estimator.inputs.pandas_input_fn(x = x_treinamento, y = y_treinamento, batch_size = 8, num_epochs = None, shuffle=True)

### Definindo função para teste

In [ ]:
# Function test
function_teste = tf.estimator.inputs.pandas_input_fn(x = x_teste, y = y_teste, batch_size = 8, num_epochs=1000, shuffle=False)

# Criando a função de regressão

In [ ]:
# Criando modelo de regressao
regressor = tf.estimator.LinearRegressor(feature_columns=colunas_x)


### Fazendo treinamento

In [ ]:
# training model, objetivo loss fuction diminua
regressor.train(input_fn=function_treinamento, steps = 10000)

### Metricas do treinamento

In [ ]:
# Metricas de treinamento
metricas_treinamento = regressor.evaluate(input_fn=function_treinamento,steps=10000)

### Metricas Teste

In [ ]:
# Testando modelo com as variaveis x_teste e y_teste
metricas_teste = regressor.evaluate(input_fn=function_teste,steps=10000)

In [ ]:
# Apresentando metricas do treinamento
metricas_treinamento

In [ ]:
# Apresentando metricas do teste
metricas_teste

### Função Previsão

In [ ]:
# Criando a função para fazer a previsao
funcao_previsao = tf.estimator.inputs.pandas_input_fn(x = x_teste,shuffle=True)

In [ ]:
# Executando a previsao
previsao = regressor.predict(input_fn=funcao_previsao)
# Visualizando os resultados previstos
#list(previsao)

### Visualizando os pesos dos estimadores de parâmetros com tensorflow
* Os estimadores de parâmetros(pesos) são utilizados para estimar a reta da regressão multípla, através deste parâmetros conseguimos estimar a reta para então utiliza-los para prever(variaveis depedentes) novos pontos(variaveis indepedentes).

In [ ]:
# Função para retornar os valores dos pesos
def list__value_param():
    name_param = regressor.get_variable_names()
    value_param = []
    name_value_param = []
    for val in name_param:
        value_param.append(regressor.get_variable_value(val))
        
    value_param = np.asarray(value_param[1:]).reshape(-1,1)# Convertendo para numpy
    name_param = np.asarray(name_param[1:]).reshape(-1,1)

    name_value_param = zip(name_param,value_param)
    name_value_param = np.asarray(name_value_param).reshape(-1,2)
    name_value_param = dict(name_value_param)
    return name_value_param

In [ ]:
# Chamando função para retornar os pesos
name_param = list__value_param()

#Apresentando as dicts
cont  = 0
for key , value in name_param.items():
    print(" B{} : {}".format(cont,value))
    cont = cont +1
    #print(key, " :: ", value)

### Inserindo os predicts em uma lista

In [ ]:
# Colocando as predicts numa lista
val_predict =[]
for p in regressor.predict(input_fn=funcao_previsao):
    val_predict.append(p['predictions'])

### Preparando os dados para visualização

In [ ]:
# Voltando para os dados sem escalonamento, utilizando função inversa 
# Inversa transformação para predict
pr = y_scal.inverse_transform(val_predict)

In [ ]:
# Convertendo do formato Series para um array
# Onde (-1,1), -1 significa para as linhas permanecerem e 1 para uma coluna
y_tes = y_teste.values.reshape(-1,1)

# Inversa transformação para predict
y_t = y_scal.inverse_transform(y_tes)

In [ ]:
# Visualizando os dados para predict
len(val_predict)#list

import numpy as np 
# Convertendo do formato Series para um array
# Onde (-1,1), -1 significa para as linhas permanecerem e 1 para uma coluna
val_predict = np.array(val_predict).reshape(-1,1)

### Visualizando o erro por mae(mean absolute error)

In [ ]:
# Utilizando mean absolute error para calcular o erro entre y_teste e o predict
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_t,pr)

In [ ]:
#Visualizando mae
mae

### Plotando o gráfico de y_teste e predict

In [ ]:
#plotando gráfico de y_teste e predict
plt.plot(y_t, 'o')
plt.plot(pr, '*', color = 'red')
plt.show()